# Зимний университет 2024. Введение в Deep learning.

Лабораторная состоит из гайда по методам глубинного обучения (Deep Learning) на Python и 3x заданий:

* [Задание 1](#Задание-1.) - реализация нейросети для классификации рукописных цифр.

* [Задание 2](#Задание-2.) - выбор подходящих аугментаций.

* [Задание 3](#Задание-3.) - реализация нейросети для устранения шумов.

Задания расположены в порядке усложнения. Среди них вы должны выполнить 1 задания обязательно, а также 2 и 3 задания по желанию. По заданию 1 на основе качества обучения сети будет составляться рейтинг: чем меньше значение функции ошибки на валидационной выборке, тем выше у вас место. Оставшееся задания можно сделать при сдаче первого - они дадут вам дополнительные баллы: второе - 3 балла, третье - 5 баллов.

Итоговый балл считается следующим образом: рассчитывается балл за первое задание (= кол-во сдавших - местро в рейтинге + 1), к нему добавляются баллы за доп задания.

По завершению каждого задания - подзывайте преподавателя для демонстрации работы.

*В этом ноутбуке изначально опущены результаты исполнения кода. Рекомендуется запускать (Shift+Enter) ячейки по мере просмотра документа*

#### Используемые модули

Обновим пакетный менеджер pip, чтобы корректно уставить необходимые модули:

In [ ]:
# При необходимости добавляйте опцию --user
!pip install --upgrade pip

Подключим модули, которые пригодятся в мини-лабораторной.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from google_drive_downloader import GoogleDriveDownloader as gdd
import os
from pathlib import Path
from sklearn.metrics import accuracy_score, confusion_matrix

import tensorflow as tf
from tensorflow.random import set_seed as set_random_seed
from keras import Model, Input
from keras.models import Sequential, load_model
from keras.datasets import fashion_mnist, mnist
from keras.utils import to_categorical
from keras.layers import (Conv2D, MaxPooling2D, Flatten, Reshape, GlobalMaxPooling2D,
                          Activation, Dense, BatchNormalization, UpSampling2D )
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import json
import os


def add_to_answer(task, answer_dict):
    """
    Добавление ответа на задание в файл с ответами.
    """
    filename = "answers.json"

    answers_data = {}
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            old_answers_data = json.load(f)
        answers_data.update(old_answers_data)

    answers_data[task] = answer_dict
    with open(filename, 'w') as f:
        json.dump(answers_data, f, indent=2)

### Нейронные сети

Если вы планируете использовать нейронные сети, имеет смысл ознакомиться с [этим](https://github.com/abidlabs/AtomsOfDeepLearning/blob/master/Atomic%20Experiments%20in%20Deep%20Learning.ipynb) ноутбуком (но для выполнения лабораторной это не обязательно).

С нейронными сетями чаще всего гораздо сложнее работать, чем с градиентным бустингом: приходится подбирать архитектуру, настраивать огромное число гиперпараметров; нет возможности узнать, по какому принципу сеть находит ответ. Однако есть ситуации, в которых нейронные сети просто не имеют адекватно работающих альтернатив:
* обработка изображений, видео - Convolutional NN;
* построение эмбеддинга с учителем и без учителя - AutoEncoder, Siamese NN;
* работа с последовательностями, в том числе генерирование - Recurrent NN и **Transformer**;
* создание генеративных моделей без учителя - Generative Adversarial Networks, Noise Conditional Score Network, и т.д.

Большая часть преимуществ достигается из-за возможности точно управлять целью обучения с помощью функции потерь, а также возможности использовать части обученной сети для решения новой задачи. Например, научив сеть предсказывать корректность словосочетания, можно применять найденный сетью способ кодирования слов для нахождения семантической близости.

#### Кратко о фреймворках

Есть несколько актуальных на данный момент фреймворков для работы с нейронными сетями:
* TensorFlow - построение статических графов вычислений, автоматический вывод градиентов, блоки для построения сетей. Имеет высокий "порог вхождения". С версии `2.0` поддерживает т.н. eager-вычисления, т.е. динамические графы. Обратная совместимость с версиями `1.*` частично отсутствует и может потреобвать переписывания части кода;
* PyTorch - построение динамических графов вычислений, лёгкий перенос вычислений на GPU (требует изменения кода);
* Keras - построение нейронных сетей из блоков. Вычисления производятся с помощью одного из низкоуровневых фреймворков (backend): TensorFlow, CNTK, Theano и др.

Если не требуется реализовывать принципиально новый алгоритм машинного обучения, внедрять наиболее современные механизмы и т.д., проще всего использовать Keras.

Для некоторых архитектур нейронных сетей пишут профильные оптимизированные решения (например, для [YOLO](https://pjreddie.com/darknet/yolo/) - алгоритма быстрого поиска объектов на изображениях реализована низкоуровневая поддержка в библиотеке [Darknet](https://github.com/pjreddie/darknet)). Фреймворки вроде TensorFlow, хоть и обеспечивающие работу с несколькими GPU в кластерах, часто проигрывают по производительности узкопрофильным решениям. На практике, если применяют TensorFlow, как правило используют [оптимизаторы](https://developer.nvidia.com/tensorrt) для ускорения инференса, а для этапа тренировки тратят приличное число ресурсов.


#### Пример

Воспользуемся фреймворком `Keras`, который работает на `Tensorflow`.

---

### Нейронные сети для обработки изображений

Загрузим датасет fashion mnist, содержащий изображения одежды (grayscale 28x28) 10 различных классов:

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
classes = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Переводим цвета из диапазона [0, 255] к диапазону [0, 1]
train_images = train_images / 255.0
test_images  = test_images / 255.0
# Переводим числовые метки в категориальные - вместо номера класса от 0 до 9 будем использовать векторы
# из 10 элементов, где все нули, кроме единицы на i-ой позиции для i-го класса.
# Такой подход позволяет избежать негативных эффектов при обучении, т.к. между классами нет строгого линейного порядка,
# как в случае с числами от 0 до 9.
train_vectors = to_categorical(train_labels)
test_vectors = to_categorical(test_labels)

Рассмотрим пример пострения нейронной сети из 2 слоев свертки, понижений размерности и вывода классов.

In [ ]:
np.random.seed(1)
set_random_seed(1)

def build_model():
    image_model = Sequential()

    image_model.add(Reshape((28, 28, 1), input_shape=(28, 28)))
    image_model.add(Conv2D(3, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    image_model.add(BatchNormalization())
    image_model.add(MaxPooling2D())

    image_model.add(Conv2D(3, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    image_model.add(BatchNormalization())
    image_model.add(MaxPooling2D())
    image_model.add(Flatten())

    image_model.add(Dense(4, activation='relu'))
    image_model.add(Dense(10, activation='softmax'))
    image_model.compile(optimizer='adam',
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])
    # Функция должна возвращать построенную и скомпилированную модель
    return image_model

Рассмотрим параметры обучения:
* `epochs` - число эпох. Эпоха заканчивается, когда сеть увидела все примеры тренировочного множества;
* `batch_size` - размер мини-батча. В [SGD](http://www.machinelearning.ru/wiki/index.php?title=Стохастический_градиентный_спуск) элементы обучающей выборки разбиваются на группы (мини-батчи) и функция потерь, вместе с её производной, рассчитывается на мини-батчах. Веса обновляются после просмотра каждого мини-батча. Вычислительно выгодней, чтобы мини-батч был достаточно большого размера, поскольку в таком случае данные будут обрабатываться параллельно;
* `validation_data` - данные для построения кривой ошибки на валидационных данных "Validation loss".

По графику функции потерь (loss) можно судить о том, имеет ли смысл обучать сеть большее число итераций.

Посмотрим на качество полученной сети:

In [ ]:
n_epochs = 10
image_model = build_model()

history = image_model.fit(
    train_images, train_vectors,
    epochs=n_epochs,
    # число примеров, на которых считается градиент:
    batch_size=32,
    validation_data=(test_images, test_vectors),
    verbose=True,  # Отключаем вывод, чтобы не перегружать ноутбук лишним выводом
)

plt.plot(range(n_epochs), (history.history['loss']), c='b')
plt.plot(range(n_epochs), (history.history['val_loss']), c='g')
plt.legend(['Train Loss', 'Validation loss'])
plt.show()

Видно, что точность крайне мала - необходимо что-то сделать с кодом сети. В следующем задании предлагается Вам возможность реализовать собственную сеть и поучаствовать с другими учениками в конкурсе за первое место в точности модели.

#### Задание 1.

Измените параметры свёрточной сети / параметры обучения (метода `fit`), чтобы повысить качество (accuracy) на **тестовых данных** до **0.87**.

Если Вы перезапустили ноутбук, надо заново загрузить датасет, выполнив начальную ячейку с кодом в разделе [Нейронные сети для обработки изображений](#Нейронные-сети-для-обработки-изображений).

In [ ]:
np.random.seed(1)
set_random_seed(1)

# Входной слой: его менять НЕ НАДО.
classification_model = Sequential()
classification_model.add(Reshape((28, 28, 1), input_shape=(28, 28)))

# далее input_shape указывать не обязательно
# Постройте свою модель на примере выше. Следует добавлять слои свертки, пулинг и нормализации признаков как приведено в этом комментарии ниже.
# classification_model.add(Conv2D(*filters_cnt*,
#                         kernel_size=*kernel_size*,
#                         strides=*strides (промежутки между элементами ядра)*,
#                         activation=*функция активации, например - 'relu'*,
#                         ))
# или
# classification_model.add(BatchNormalization())
# или
# classification_model.add(MaxPooling2D())


classification_model.add(...)
classification_model.add(...)
classification_model.add(...)
# .... и еще слои, сколько посчитаете нужным


# Слой ниже (и последующие слои) менять НЕ надо
classification_model.add(Flatten(name='flatten'))
classification_model.add(Dense(4, activation='relu'))
classification_model.add(Dense(10, activation='softmax'))

In [ ]:
classification_model.compile(optimizer='adam',
                             loss='categorical_crossentropy',
                             metrics=['accuracy'])

classification_model.fit(train_images, train_vectors, epochs=5, batch_size=128,
                         validation_data=(test_images, test_vectors))

In [ ]:
def evaluate_model_for_task_2(model):
    test_acc = accuracy_score(test_labels, np.argmax(model.predict(test_images), axis=-1))
    print(f"Качество: {test_acc}")
    print("Тест на качество {}".format("НЕ пройден :(" if 0.87 > test_acc else "пройден :)"))

Вызовите функцию `evaluate_model_for_task_2`, передав ей обученную модель, чтобы увидеть качество этой модели. При сдаче задания качество должно быть не ниже **0.88**.

Не забудьте сохранить модель, выполнив ячейки ниже.

_Примечание: модель, полученная в ходе выполнения этого задания, будет использована далее, поэтому важно, чтобы она называлась_ `classification_model`

In [91]:
#  classification_model = best_model
evaluate_model_for_task_2(classification_model)

Качество: 0.8831
Тест на качество пройден :)


Модель понадобится в следующем задании, поэтому сохраним её

In [ ]:
second_task_saved_model = os.path.join("models", "task_2_weights.h5")
# Создаём папку, куда сохраним результаты, если папки ещё нет
Path("models").mkdir(parents=True, exist_ok=True)
classification_model.save(second_task_saved_model)

add_to_answer(
    "task_2",
    {
        "model_path": second_task_saved_model,
    },
)

---

Посмотрим как выглядит предсказание:

In [ ]:
np.random.seed(123456)
test_ind = np.random.randint(test_images.shape[0])
test_image = test_images[test_ind]
test_vector = test_vectors[test_ind]
predicted_test_vector = classification_model.predict(test_image[np.newaxis]).reshape((10,))

plt.imshow(test_image, cmap='gray')
plt.title('Тестовое изображение')
plt.figure()

fig, ax = plt.subplots(1, 2, figsize=(17, 4))
ax[0].bar(range(10), test_vector, tick_label=classes)
ax[1].bar(range(10), predicted_test_vector, tick_label=classes)
for i, title in enumerate(['Правильный вектор классов', 'Предсказанный вектор']): ax[i].set_title(title)

Получим предсказания для первых 100 изображений:

In [ ]:
%time
predicted_vectors = classification_model.predict(test_images[:100]).reshape((100, 10))

Посмотрим на каких изображениях сеть ошибается:

In [ ]:
predicted_labels = np.argmax(predicted_vectors, axis=1)  # предсказанные индексы классов - где верояность наибольшая

pred = test_labels[:100] != predicted_labels
misclassified_images = test_images[:100][pred]
misclassified_correct = test_labels[:100][pred]
misclassified_predicted = predicted_labels[:100][pred]

cm = confusion_matrix(test_labels[:100], predicted_labels[:100])
plt.imshow(cm, interpolation='nearest')
plt.xticks(range(10), classes, rotation=90)
plt.yticks(range(10), classes)
plt.colorbar()
plt.title("Confusion matrix")
plt.figure()

fig, ax = plt.subplots(2, 6, figsize=(15, 4))
for i in range(2 * 6):
    ax.flatten()[i].imshow(misclassified_images[i], cmap='gray')
    ax.flatten()[i].axis('off')
    ax.flatten()[i].set_title(f"{classes[misclassified_predicted[i]]} / {classes[misclassified_correct[i]]}")

---

## Аугментация

Часто доступные на этапе обучения размеченные данные оказываются смещёнными, не целиком покрывающими допустимое для данной задачи распределение входных данных.
Также, при использовании нейронных сетей с большим числом параметров, на этапе обучения необходимо большое число различных данных, чтобы модель не деградировала до "табличной функции", запоминая все входные данные.

Для борьбы с обеими проблемами хорошо подходит подход искусственного расширения обучающего набора данных – аугментация. Идея заключается в применении случайных трансформаций к каждому отдельному изображению, при сохранении меток классов. Таким образом можно получить датасет произвольного размера и добиться от нейронной сети определённых свойств (разумеется, для фиксированного, ограниченного распределения входных данных), например: инвариантности к повороту объекта – за счёт обучения на всевозможных поворотах входных изображений.

Важно, что выбор набора трансформаций и распределений из которых генерируются их параметры, специфичен для каждой задачи. Например, при классификации деревьев не разумно выполнять повороты на $\pi$, поскольку известно, что деревья всегда растут снизу вверх.

---

Будем загружать датасет из GoogleDisk, воспользуемся библиотекой `googledrivedownloader`.

Скачаем файлы с данными:

In [ ]:
ids_to_file_names = {
    '1iFK4-fCqs8l5lWmYnUYlkv28GDb43FUv': 'fashion_images.npy',
    '15NUcK_Pg4iEUttK1XiG0SonJaEbaaVnI': 'fashion_labels.npy',
    '1AXtbCz3EE2xVuUrGgEco7a-lMh8Vqm0o': 'val_fashion_images.npy',
    '1fk1mZxBLRryWp4IoLHRr5IwiaKy4WL_Y': 'val_fashion_labels.npy',
    '1tWy1d9lkrCzkiSZBlfd1G6YV4sZ7pNt4': 'test_fashion_images.npy',
}

# Файлы будем загружать и сохранять в папку data
for _id in ids_to_file_names:
    ids_to_file_names[_id] = os.path.join("data", ids_to_file_names[_id])

# Создадим папку data, если её ещё нет
Path("data").mkdir(parents=True, exist_ok=True)


for file_id in ids_to_file_names:
    # Качаем файл только если он ещё не скачан
    if not os.path.isfile(ids_to_file_names[file_id]):
        gdd.download_file_from_google_drive(file_id=file_id,
                                            dest_path=os.path.join('.', ids_to_file_names[file_id]),
                                            unzip=True)

Загрузим данные – изображения 64x64 с цветным шумом и случайно аффинно трансформированными снимками одежды:

In [ ]:
def load_from_data(fname):
    return np.load(os.path.join("data", fname))

fashion_images = load_from_data("fashion_images.npy")
fashion_labels = load_from_data("fashion_labels.npy")

val_fashion_images = load_from_data("val_fashion_images.npy")
val_fashion_labels = load_from_data("val_fashion_labels.npy")

test_fashion_images = load_from_data("test_fashion_images.npy")

В качестве тренировочного и валидационного датасета даны изображения и метки (вектора вероятностей классов одежды), в качестве тестового – только изображения.

Ниже дана модель, гиперпараметры которой (количества каналов, размеры ядер, функции активации, архитектуру и т.п.) менять **нельзя**.

In [ ]:
def make_fashion_model():
    imodel = Sequential()
    imodel.add(Reshape((64, 64, 3), input_shape=(64, 64, 3)))
    imodel.add(Conv2D(4, kernel_size=(3, 3), strides=(1, 1),
                         activation='relu', padding='same'))
    imodel.add(BatchNormalization())
    imodel.add(MaxPooling2D())
    imodel.add(Conv2D(8, kernel_size=(3, 3), strides=(1, 1),
                         activation='relu', padding='same'))
    imodel.add(BatchNormalization())
    imodel.add(MaxPooling2D())
    imodel.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1),
                         activation='relu', padding='same'))
    imodel.add(BatchNormalization())
    imodel.add(MaxPooling2D())
    imodel.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
                         activation='relu', padding='same'))
    imodel.add(BatchNormalization())
    imodel.add(MaxPooling2D())
    imodel.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1),
                         activation='relu', padding='same'))
    imodel.add(BatchNormalization())
    imodel.add(MaxPooling2D())
    imodel.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1),
                         activation='relu', padding='same'))
    imodel.add(BatchNormalization())
    imodel.add(Flatten())

    iclf = Sequential()
    iclf.add(imodel)
    iclf.add(Dense(10, activation='softmax'))

    iclf.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    return iclf

---

#### Задание 2 (дополнительное).

**Внимание!** Это задание повышенной сложности, за его решение даётся 2.5 балла, выполнять его необязательно.

Найти такие параметры аугментации (ниже), чтобы качество классификации (accuracy) на **тестовых** данных было **не ниже 0.6**.

*Настоятельно рекомендуется сначала внимательно изучить обучающую и тестовую выборки, а не подбирать параметры случайным образом.*

In [ ]:
train_fashion_images = fashion_images.copy()
# здесь можно сделать предварительную обработку всего обучающего датасета
train_fashion_labels = fashion_labels

In [ ]:
# здесь следует изменить параметры аугментации (параметры ImageDataGenerator):
print(train_fashion_images[0])
aug_params = {  # Параметры задаём сначала в словаре, чтобы их потом сохранить
    "featurewise_center": False,  # Bool: False/True
    "samplewise_center": False,  # Bool: False/True
    "featurewise_std_normalization": False,  # Bool: False/True
    "samplewise_std_normalization": False,  # Bool: False/True
    "width_shift_range": 0.0, # integer pos number, but in float type (idk)
    "height_shift_range": 0.0, # integer pos number, but in float type (idk)
    "brightness_range": None, # No need to change
    "shear_range": 0,
    "zoom_range": 0, # Float pos number
    "channel_shift_range": 0,
    "fill_mode": 'nearest',
    "cval": 0.0,
    "horizontal_flip": False,  # Bool: False/True
    "vertical_flip": False,  # Bool: False/True
    "rescale": None,
}

aug = ImageDataGenerator(**aug_params)

Из параметров обучения можно менять `batch_size` и `epochs`, но использовать можно не больше 200 эпох.

При желании можно разобраться в механизме callback'ов и передать в `fit` дополнительные параметры для автоматического сохранения весов модели к состоянию на "лучшей" итерации обучения (лучшей с точки зрения качества на валидационных данных), но это делать не обязательно.

In [ ]:
fit_params = {
    "batch_size": 32,
    "epochs": 200,
}

fashion_clf = make_fashion_model()

aug.fit(train_fashion_images)
gen = aug.flow(train_fashion_images, train_fashion_labels,
               batch_size=fit_params["batch_size"])

print(gen[0][0])

hist = fashion_clf.fit(
    gen,
    steps_per_epoch=len(train_fashion_images) // fit_params["batch_size"],
    validation_data=(val_fashion_images, val_fashion_labels),
    **fit_params,
)

In [ ]:
def plot_accuracy(history):
    acc = 'acc' if 'acc' in history else 'accuracy'
    val_acc = 'val_' + acc

    plt.plot(range(len(history[acc])), history[acc], color='b')
    plt.plot(range(len(history[val_acc])), history[val_acc], color='g')

    return hist.history[val_acc][-1]

plot_accuracy(hist.history)

Для проверки сохраняются веса сети:

In [ ]:
aug_saved_model = os.path.join("models", "task_4_weights.h5")
# Создаём папку, куда сохраним результаты, если папки ещё нет
Path("models").mkdir(parents=True, exist_ok=True)
fashion_clf.save(aug_saved_model)

add_to_answer(
    "task_4",
    {
        "model_path": aug_saved_model,
        "fit_params": fit_params,
        "aug_params": aug_params,
    },
)

---

#### Задание 3.

**Задание повышенной сложности!**
Необходимл реализовать код шумоподавляющего автокодировщика.

Для начала загрузим данные датасета MNIST.

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

Метки классов можно отбросить - для создания модели удаления шумов они не нужны. Сохраним как y_train и y_test точные снимки рукописных цифр, а в x_train и x_test добавим пуассоновские шумы.

In [ ]:
y_train, y_test = x_train.astype("float32") / 255.0, x_test.astype("float32") / 255.0

x_train = np.sum(
                        [
                            x_train.astype("float32") / 255.0,
                            (np.random.poisson(64,x_train.shape[0] * x_train.shape[1] * x_train.shape[2]).reshape(x_train.shape)) / 255.0,
                        ],
                        axis=0,
                    ).astype("float32")

x_test = np.sum(
                        [
                            x_test.astype("float32") / 255.0,
                            (np.random.poisson(64,x_test.shape[0] * x_test.shape[1] * x_test.shape[2]).reshape(x_test.shape)) / 255.0,
                        ],
                        axis=0,
                    ).astype("float32")

x_train, x_test = x_train / np.amax(x_train), x_test / np.amax(x_test)

Посмотрим как теперь выглядят шумные и точные данные

In [ ]:
np.random.seed(123456)
test_ind = np.random.randint(y_test.shape[0])
test_noisy = x_test[test_ind]
test_clear = y_test[test_ind]

plt.imshow(test_noisy, cmap='gray')
plt.title('Тестовое изображение (шумное)')
plt.figure()

plt.imshow(test_clear, cmap='gray')
plt.title('Тестовое изображение (точное)')
plt.figure()

Ниже вам пердоставляется возможность построить модель для проведения обешумливания. Поскольку быстрорасчетных численных метрик для оценки точности изображений не так много, как для оценки классификаиции, то вам предоставляется оценить обесшумлевание "на глаз".

Не забудьте, что автокодировщик должен состоять из Encoder (кодировщика) и Decoder (декодировщика).

In [ ]:
def build_model():
  denoise_model = Sequential()
  denoise_model.add(Reshape((28, 28, 1), input_shape=(28, 28)))

  # далее input_shape указывать не обязательно
  # Постройте свою модель на примере выше. Следует добавлять слои свертки, пулинг и нормализации признаков как приведено в этом комментарии ниже.
  # denoise_model.add(Conv2D(*filters_cnt*,
  #                         kernel_size=*kernel_size*,
  #                         strides=*strides (промежутки между элементами ядра)*,
  #                         activation=*функция активации, например - 'relu'*,
  #                         padding='same' (Писать именно так! поскольку свертка изменяет размер, а на выходе ожидаем ту же размерность, что и на входе)
  #                         ))
  # или
  # denoise_model.add(BatchNormalization())
  # или
  # denoise_model.add(MaxPooling2D())
  # или
  # denoise_model.add(MaxPooling2D())
  # или
  # denoise_model.add(UpSampling2D())


  # encoder
  # denoise_model.add(...)
  # denoise_model.add(...)
  # denoise_model.add(...)

  # decoder
  # denoise_model.add(...)
  # denoise_model.add(...)
  # denoise_model.add(...)
  # .... и еще слои, сколько посчитаете нужным


  # Слой ниже (и последующие слои) менять НЕ надо
  denoise_model.add(Conv2D(1, kernel_size=(1, 1), strides=(1, 1), padding='same', activation='relu'))
  denoise_model.compile(optimizer='adam',
                        loss='mse')
  return denoise_model

In [ ]:
denoise_model = build_model()
n_epochs = 100

history = denoise_model.fit(
    x_train, y_train,
    epochs=n_epochs,
    # число примеров, на которых считается градиент:
    batch_size=32,
    validation_data=(x_test, y_test),
    verbose=True,
)

plt.plot(range(n_epochs), (history.history['loss']), c='b')
plt.plot(range(n_epochs), (history.history['val_loss']), c='g')
plt.legend(['Train Loss', 'Validation loss'])
plt.show()